<a href="https://colab.research.google.com/github/NahidurRahaman/Machine_Learning/blob/main/hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install -q torch torchvision timm mlflow pyngrok pytorch-lightning==2.4.0
!pip install -q "huggingface-hub<1.0" "transformers<4.44" torchmetrics==1.4.0 scikit-learn numpy matplotlib seaborn

# CLEAN CACHE (optional but recommended for fresh installs/updates)
import os, shutil
if os.path.exists("/root/.cache/huggingface/hub"):
    shutil.rmtree("/root/.cache/huggingface/hub")

# ===============================
# 📁 System & File Handling
# ===============================
import os                    # OS operations (paths, env vars)
import shutil                # High-level file operations
from pathlib import Path     # Clean path handling

# ===============================
# 🖼 Image Processing
# ===============================
from PIL import Image        # Image loading & processing

# ===============================
# 🔥 PyTorch Core
# ===============================
import torch                 # Core PyTorch
from torch.utils.data import DataLoader

# ===============================
# ⚡ PyTorch Lightning
# ===============================
import pytorch_lightning as pl

# ===============================
# 🧠 Vision & Transforms
# ===============================
from torchvision import datasets, transforms
import timm                  # Pretrained vision models (EfficientNet, ViT, ConvNeXt)

# ===============================
# 📊 Experiment Tracking (MLflow)
# ===============================
import mlflow
import mlflow.pytorch

# ===============================
# 🌐 Public MLflow UI (ngrok)
# ===============================
from pyngrok import ngrok

In [ ]:
# This line checks whether a GPU (CUDA) is available.
# If a GPU exists, training will run on the GPU for much faster performance.
# If no GPU is found (e.g., running on CPU-only machine), it will fall back to CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print which device the model will use
print("Running on:", device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# =============================
# 🔧 USER CONFIGURATION
# (Edit these values according to your project)
# =============================

# 👉 Your Ngrok authentication token (required to create a public URL for MLflow UI)
# ⚠️ Keep this private. Anyone with this token can use your ngrok account.
NGROK_AUTH_TOKEN = "36CLviJ3eedsGnzgPHm7MM9g4De_4tDSxH3CgGEQCytxcXKgX"

# 👉 Directory where MLflow will store run logs, metrics, parameters, and model artifacts
MLRUNS_DIR = "/content/drive/MyDrive/Project-1/mlruns"

# 👉 Path to your dataset ZIP file (usually stored in Google Drive)
# ML pipeline will unzip and prepare the dataset from here.
DATA_ZIP = "/content/drive/MyDrive/Project-1/archive.zip"

# 👉 Directory where the dataset should be extracted
EXTRACT_DIR = "/content/drive/MyDrive/Project-1/RiceLeafsDisease"

# 👉 Image size for training (input size for vision models like ViT, EfficientNet, ResNet)
IMG_SIZE = 224

# 👉 Number of images per batch during training
# Higher batch size → faster training but requires more GPU memory
BATCH_SIZE = 32

# 👉 Maximum number of epochs the model will train
# One epoch = one full pass over the entire dataset
MAX_EPOCHS = 15

In [ ]:
import os  # File/folder operations (join paths, check existence)
import pytorch_lightning as pl  # Lightning framework for structured training
from torch.utils.data import DataLoader  # Wraps datasets into batches with shuffle & workers
from torchvision import datasets, transforms  # Dataset loading & image preprocessing

# ----------------------------------------
# LightningDataModule for Plant Disease Dataset
# ----------------------------------------
class PlantDataModule(pl.LightningDataModule):

    def __init__(self, data_dir, batch_size=32, img_size=224):
        """
        Initialize the DataModule.
        Args:
            data_dir (str): Root path to dataset
            batch_size (int): Images per batch
            img_size (int): Resize images to square size
        """
        super().__init__()  # Initialize LightningDataModule
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size

        # -------------------------------
        # Transformations for training images
        # -------------------------------
        self.train_tfms = transforms.Compose([
            transforms.Resize((img_size, img_size)),  # Resize all images
            transforms.RandomHorizontalFlip(),        # Data augmentation: random horizontal flip
            transforms.ToTensor()                     # Convert PIL Image → PyTorch Tensor
        ])

        # -------------------------------
        # Transformations for validation/test images
        # -------------------------------
        self.val_tfms = transforms.Compose([
            transforms.Resize((img_size, img_size)),  # Resize all images
            transforms.ToTensor()                     # Convert PIL Image → PyTorch Tensor
        ])

    # -------------------------------
    # Setup datasets (train/val/test)
    # -------------------------------
    def setup(self, stage=None):
        """
        Called by Lightning at the start of training/validation/testing.
        Loads datasets using ImageFolder.
        """
        train_dir = os.path.join(self.data_dir, "train")  # Path to training images
        val_dir = os.path.join(self.data_dir, "validation")    # Path to validation images
        test_dir = os.path.join(self.data_dir, "validation")    # Path to test images

        # Safety check: training folder must exist
        if not os.path.exists(train_dir):
            raise FileNotFoundError(f"Expected train folder at {train_dir}")

        # Load datasets
        self.train_ds = datasets.ImageFolder(train_dir, transform=self.train_tfms)
        self.val_ds = datasets.ImageFolder(val_dir, transform=self.val_tfms) if os.path.exists(val_dir) else None
        self.test_ds = datasets.ImageFolder(test_dir, transform=self.val_tfms) if os.path.exists(test_dir) else None

    # -------------------------------
    # Training DataLoader
    # -------------------------------
    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            shuffle=True,  # Shuffle each epoch for better generalization
            num_workers=2  # Parallel data loading
        )

    # -------------------------------
    # Validation DataLoader
    # -------------------------------
    def val_dataloader(self):
        return DataLoader(
            self.val_ds,
            batch_size=self.batch_size,
            num_workers=2
        ) if self.val_ds else None  # Returns None if no validation dataset

    # -------------------------------
    # Test DataLoader
    # -------------------------------
    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            num_workers=2
        ) if self.test_ds else None  # Returns None if no test dataset

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import timm
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics import Accuracy


class TomatoHybridModel(pl.LightningModule):
    def __init__(self, num_classes=6, lr=1e-4, weight_decay=1e-5):
        super().__init__()
        self.save_hyperparameters()

        # Backbone models
        self.swin = timm.create_model(
            "swin_tiny_patch4_window7_224",
            pretrained=True,
            features_only=True,
            out_indices=[3]
        )

        self.vit = timm.create_model(
            "vit_base_patch16_224",
            pretrained=True,
            num_classes=0
        )

        self.efficientnet = timm.create_model(
            "efficientnetv2_rw_s",
            pretrained=True,
            num_classes=0
        )

        # Fusion head
        self.fusion = nn.Sequential(
            nn.LazyLinear(1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

        # Loss & metrics
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
        self.train_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.val_acc   = Accuracy(task="multiclass", num_classes=num_classes)

        # 🔥 History storage
        self.train_acc_history = []
        self.val_acc_history = []
        self.train_loss_history = []
        self.val_loss_history = []

    def forward(self, x):
        swin_feat = self.swin(x)[-1].mean([2, 3])
        vit_feat  = self.vit(x)
        eff_feat  = self.efficientnet(x)
        if eff_feat.dim() == 4:
            eff_feat = eff_feat.mean([2, 3])

        fused = torch.cat([swin_feat, vit_feat, eff_feat], dim=1)
        return self.fusion(fused)

    def training_step(self, batch, _):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = self.train_acc(logits, y)

        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)

        return loss

    def validation_step(self, batch, _):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = self.val_acc(logits, y)

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    # 🔥 Epoch end hooks
    def on_train_epoch_end(self):
        self.train_acc_history.append(self.train_acc.compute().item())
        self.train_loss_history.append(
            self.trainer.callback_metrics["train_loss"].item()
        )
        self.train_acc.reset()

    def on_validation_epoch_end(self):
        self.val_acc_history.append(self.val_acc.compute().item())
        self.val_loss_history.append(
            self.trainer.callback_metrics["val_loss"].item()
        )
        self.val_acc.reset()

    def configure_optimizers(self):
        optimizer = AdamW(
            self.parameters(),
            lr=self.hparams.lr,
            weight_decay=self.hparams.weight_decay
        )
        scheduler = ReduceLROnPlateau(
            optimizer, mode="min", patience=3, factor=0.5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"}
        }

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    min_delta=0.001,
    mode="min",
    verbose=True
)

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="/content/drive/MyDrive/Project-1/checkpoints",
    filename="best_model",   # 🔥 .ckpt নিজে যোগ হবে
    save_top_k=1,
    mode="min",
    verbose=True
)


In [ ]:
dm = PlantDataModule(EXTRACT_DIR, batch_size=BATCH_SIZE, img_size=224)

try:
    dm.setup()
except Exception as e:
    raise RuntimeError(f"Data setup failed: {e}")

num_classes = len(dm.train_ds.classes)
print("Classes:", dm.train_ds.classes)

In [ ]:
print("Preparing data module and model (this will error if your dataset path is wrong)...")
dm = PlantDataModule(EXTRACT_DIR, batch_size=BATCH_SIZE, img_size=224)

try:
    dm.setup()
except Exception as e:
    raise RuntimeError(f"Data setup failed: {e}")

num_classes = len(dm.train_ds.classes)
print("Classes:", dm.train_ds.classes)

model = TomatoHybridModel(num_classes=num_classes)

In [ ]:
checkpoints = os.path.join(
    os.getcwd(), "checkpoints", "/content/drive/MyDrive/Project-1/best_model.pth"
)

In [ ]:
if os.path.exists(checkpoints):
    model.load_state_dict(torch.load(checkpoints))
    print("Model loaded successfully from checkpoints.")

In [ ]:
trainer = pl.Trainer(
    max_epochs=15,
)

In [ ]:
trainer.fit(model, dm)

In [ ]:
# SAVE MODEL HERE
MODEL_PATH = "/content/drive/MyDrive/Project-1/best_model.pth"
# Use the model's state_dict() for saving the raw weights
torch.save(model.state_dict(), MODEL_PATH)
print("Model saved at:", MODEL_PATH)

num_classes = len(dm.train_ds.classes)
num_classes

In [ ]:
trainer.validate(model, dm)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you collected per-epoch averages — or just plot what you have
plt.figure(figsize=(12, 5))

# Get max loss and accuracy values for y-axis limits
max_loss = max(model.train_loss_history + model.val_loss_history)
max_acc = max(model.train_acc_history + model.val_acc_history)

plt.subplot(1, 2, 1)
plt.plot(model.train_loss_history,marker='o', linewidth=2, label="Train Loss")
plt.plot(model.val_loss_history,   marker='o', linewidth=2, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.ylim(-0.02, 1.02)
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(model.train_acc_history, marker='o', linewidth=2, label="Train Accuracy")
plt.plot(model.val_acc_history,   marker='o', linewidth=2, label="Val Accuracy")
plt.ylim(0, 1.02)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)


plt.tight_layout()
plt.show()

In [ ]:
import torch
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

model.eval()
model.to('cuda' if torch.cuda.is_available() else 'cpu')

all_preds = []
all_labels = []

test_dl = dm.test_dataloader()
if test_dl is None:
    print("No test set found → using validation set")
    test_dl = dm.val_dataloader()

with torch.no_grad():
    for batch in test_dl:
        x, y = batch
        x = x.to(model.device)
        logits = model(x)
        preds = logits.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

# Create & plot
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dm.train_ds.classes)

plt.figure(figsize=(10, 8))
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix (Test set)")
plt.tight_layout()
plt.show()

# Or with seaborn (nicer for many classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=dm.train_ds.classes,
            yticklabels=dm.train_ds.classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
!pip install grad-cam

In [ ]:
# ───────────────────────────────────────────────────
#  Grad-CAM visualization for 3 images (SwinV2 + Lightning DataModule)
# ──────────────────────────────────────────────────

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# ──────────────────────────────────────────────
#  Configuration
# ──────────────────────────────────────────────
image_paths = [
    "/content/drive/MyDrive/BongoDev/Project-1/na1.jpg",
    "/content/drive/MyDrive/BongoDev/Project-1/na.JPG",
    "/content/drive/MyDrive/BongoDev/Project-1/na2.JPG",
]
image_labels = ["Sample 1", "Sample 2", "Sample 3"]

# Use patch embedding conv as target layer (SwinV2)
# Corrected: TomatoHybridModel does not have a 'backbone' attribute,
# instead it has specific components like 'swin', 'vit', 'efficientnet'.
# We target a layer within the 'swin' component for Grad-CAM.
target_layers = [model.swin.patch_embed.proj]

# Device
device = next(model.parameters()).device
model.to(device).eval()

# Grad-CAM object
cam = GradCAM(model=model, target_layers=target_layers)

# Validation transform
val_transform = dm.val_tfms  # or define manually if dm is not defined

# ──────────────────────────────────────────────
#  Visualization setup
# ──────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 6), dpi=130)
fig.suptitle(
    "Grad-CAM: Where the Model is Looking (Arrow = Strongest Attention)",
    fontsize=16, fontweight='bold', y=1.04
)

for idx, (img_path, label) in enumerate(zip(image_paths, image_labels)):
    ax = axes[idx]
    try:
        # Load image
        img_pil = Image.open(img_path).convert("RGB")
        img_tensor = val_transform(img_pil).unsqueeze(0).to(device)

        # Predict
        with torch.no_grad():
            logits = model(img_tensor)
            pred_class_idx = logits.argmax(dim=1).item()
            pred_class_name = dm.train_ds.classes[pred_class_idx]
            confidence = torch.softmax(logits, dim=1)[0, pred_class_idx].item()

        # Grad-CAM
        targets = [ClassifierOutputTarget(pred_class_idx)]
        grayscale_cam = cam(input_tensor=img_tensor, targets=targets)[0, :]

        # Overlay heatmap (normalize correctly)
        img_np = np.array(img_pil.resize((dm.img_size, dm.img_size))) / 255.0
        visualization = show_cam_on_image(img_np, grayscale_cam, use_rgb=True)

        # Find hotspot
        max_y, max_x = np.unravel_index(np.argmax(grayscale_cam), grayscale_cam.shape)

        # Plot image + heatmap
        ax.imshow(visualization)

        # Draw arrow safely
        arrow_offset = 60
        ax.annotate(
            "",
            xy=(max_x, max_y),
            xytext=(min(max_x + arrow_offset, dm.img_size-1), max(max_y - arrow_offset, 0)),
            arrowprops=dict(
                facecolor='red',
                edgecolor='darkred',
                width=4,
                headwidth=14,
                headlength=18,
                shrink=0.08,
                alpha=0.9,
                linewidth=1.5,
                zorder=10
            )
        )

        # Hotspot marker
        ax.plot(max_x, max_y, marker='o', markersize=14,
                color='red', markeredgecolor='white', markeredgewidth=2.5, zorder=11)

        # Title with prediction info
        ax.set_title(f"{label}\nPred: {pred_class_name}\nConf: {confidence:.1%}",
                     fontsize=11, pad=12, fontweight='medium')
        ax.axis('off')

    except Exception as e:
        ax.text(0.5, 0.5, f"Error\n{str(e)[:60]}...", ha='center', va='center', color='red', fontsize=10)
        ax.axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

In [ ]:
import numpy as np
import torch
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

# Ensure model is in evaluation mode and on the correct device
model.eval()
# Get the device the model is currently on (e.g., 'cuda' or 'cpu')
model_device = next(model.parameters()).device
model.to(model_device)

all_labels = []
all_logits = [] # Collect raw logits for ROC curve calculation

# Determine which dataloader to use (validation or test)
# The previous confusion matrix plot used test_dataloader first, then val_dataloader as fallback.
current_dataloader = dm.test_dataloader()
if current_dataloader is None:
    print("No test set found, using validation set for ROC curve calculation.")
    current_dataloader = dm.val_dataloader()
    if current_dataloader is None:
        raise ValueError("Neither validation nor test dataloader is available. Cannot compute ROC curve.")
else:
    print("Using test set for ROC curve calculation.")

with torch.no_grad(): # Disable gradient calculations for inference
    for inputs, targets in current_dataloader:
        inputs = inputs.to(model_device)
        logits = model(inputs)
        all_logits.append(logits.cpu())  # Move logits to CPU before appending
        all_labels.append(targets.cpu()) # Move targets to CPU before appending

# Concatenate all collected tensors into single NumPy arrays
all_logits = torch.cat(all_logits).numpy()
all_labels = torch.cat(all_labels).numpy()

# Get number of classes from the data module
num_classes = len(dm.train_ds.classes)
class_names = dm.train_ds.classes # For plotting legends

# Binarize the true labels (one-hot encode) for multiclass ROC calculation
y_true = label_binarize(all_labels, classes=list(range(num_classes)))

# Convert logits to probabilities using softmax for y_score
y_score = torch.softmax(torch.tensor(all_logits), dim=1).numpy()

print(f"Collected {len(all_labels)} samples for ROC analysis.")
print(f"Shape of y_true: {y_true.shape}")
print(f"Shape of y_score: {y_score.shape}")


In [ ]:
plt.figure(figsize=(8, 6))

for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_true[:, i], y_score[:, i])
    roc_auc = auc(fpr, tpr)

    plt.plot(
        fpr,
        tpr,
        label=f"Class {i} (AUC = {roc_auc:.3f})"
    )

plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Multiclass ROC Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
import torch

# ===============================================
# Step 1: Collect predictions & true labels on test set
# ===============================================
model.eval()
all_preds = []
all_labels = []
all_probs = []  # optional for ROC if needed later

test_dl = dm.test_dataloader()  # or dm.val_dataloader() if no test set

with torch.no_grad():
    for batch in test_dl:
        x, y = batch
        x = x.to(model.device)
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        preds = logits.argmax(dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(y.cpu().numpy())
        all_probs.extend(probs)

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_probs = np.array(all_probs)

class_names = dm.train_ds.classes  # your 10 class names

# ===============================================
# Step 2: Print Classification Report (Precision, Recall, F1)
# ===============================================
print("\n" + "="*60)
print("Classification Report (Precision, Recall, F1-Score)")
print("="*60)
report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)
print(report)

# ===============================================
# Step 3: Per-class Precision, Recall, F1 Bar Plot
# ===============================================
precision, recall, f1, support = precision_recall_fscore_support(all_labels, all_preds, average=None)

plt.figure(figsize=(14, 6))
x = np.arange(len(class_names))
width = 0.25

plt.bar(x - width, precision, width, label='Precision', color='skyblue')
plt.bar(x, recall, width, label='Recall', color='lightgreen')
plt.bar(x + width, f1, width, label='F1-Score', color='salmon')

plt.xlabel('Classes')
plt.ylabel('Score')
plt.title('Per-Class Precision, Recall & F1-Score')
plt.xticks(x, class_names, rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# ===============================================
# Step 4: Confusion Matrix (helps understand precision errors)
# ===============================================
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

plt.figure(figsize=(12, 10))
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix (Test Set)")
plt.tight_layout()
plt.show()

# ===============================================
# Step 5: Macro & Weighted Average Summary
# ===============================================
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

print("\n" + "="*60)
print("Summary Averages")
print("="*60)
print(f"Macro Average    → Precision: {precision_macro:.4f} | Recall: {recall_macro:.4f} | F1: {f1_macro:.4f}")
print(f"Weighted Average → Precision: {precision_weighted:.4f} | Recall: {recall_weighted:.4f} | F1: {f1_weighted:.4f}")